In [1]:
print("Ok")

Ok


In [2]:
%pwd

'c:\\Users\\user\\Desktop\\Care-Reach-Medical-chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\user\\Desktop\\Care-Reach-Medical-chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [7]:
extracted_data=load_pdf_file(data='Data/')

In [9]:
# extracted_data

In [10]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 6970


In [13]:
# text_chunks

In [25]:
from langchain.embeddings import HuggingFaceEmbeddings

In [26]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [27]:
embeddings = download_hugging_face_embeddings()

In [28]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [30]:
# query_result

In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [15]:
import os
from pinecone import Pinecone, ServerlessSpec

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

# Create an instance of Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "carebot"  # Use lowercase letters only

# Create index if it doesn't exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print(f"Index '{index_name}' created successfully.")
else:
    print(f"Index '{index_name}' already exists.")


Index 'carebot' already exists.


In [ ]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [30]:
from langchain.vectorstores import Pinecone

# Embed each chunk and upsert the embeddings into your Pinecone index
docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)


NameError: name 'embeddings' is not defined

In [31]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

ImportError: cannot import name 'PineconeVectorStore' from 'langchain_pinecone' (c:\Users\user\miniconda3\envs\carebot\lib\site-packages\langchain_pinecone\__init__.py)

In [32]:
docsearch

NameError: name 'docsearch' is not defined

In [33]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

NameError: name 'docsearch' is not defined

In [34]:
retrieved_docs = retriever.invoke("What is Acne?")

NameError: name 'retriever' is not defined

In [35]:
retrieved_docs

NameError: name 'retrieved_docs' is not defined

In [36]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [37]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [38]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

NameError: name 'llm' is not defined

In [39]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

NameError: name 'rag_chain' is not defined

In [40]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

NameError: name 'rag_chain' is not defined